<a href="https://colab.research.google.com/github/linhkid/gdg-codelab-25/blob/main/GDG_Gemma2.0_multiagent_funccall_vertex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codelab: Build your first agentic AI with Gemma 2.0 and Vertex AI

# Step 1: Setup and Authentication

In [44]:
# Install dependencies and authenticate with Vertex AI

# @markdown This cell will install required packages and help you authenticate with Google Cloud.

!pip install -q -U google-cloud-aiplatform
!pip install -q matplotlib pandas numpy

from google.colab import auth
import os
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown

# Authenticate to Google Cloud
auth.authenticate_user()

print("✅ Required packages installed. Authenticating with Vertex AI...\n")

✅ Required packages installed. Authenticating with Vertex AI...



# Step 2: Initialize Vertex AI for Gemma 2.0

In [47]:
# Initialize Vertex AI and Gemma 2.0
# @markdown Configure your project and set up Gemma 2.0 via Vertex AI

import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel

# Set your project ID
PROJECT_ID = "gdg-codelab-12thMay "  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    try:
        # Try to retrieve project ID from environment variable
        PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
        print(f"Using project ID from environment: {PROJECT_ID}")
    except:
        print("❌ Please set your Google Cloud project ID")

# Set location
LOCATION = "asia-southeast1"  # Vertex AI region

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Gemma 2.0 model setup
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/models/gemma-2-2b-it-1742823574849"  # Instruction-tuned Gemma 2.0 model

try:
    # Load the Gemma 2.0 model from Vertex AI
    model = GenerativeModel(MODEL_NAME)
    print(f"✅ Successfully initialized {MODEL_NAME} on Vertex AI")
except Exception as e:
    print(f"❌ Error initializing model: {e}")
    print("Please check your project configuration and model availability in Vertex AI")

# Helper function for generating responses
def generate_response(prompt, temperature=0.2, max_output_tokens=1024, top_p=0.8):
    """Generate a response from Gemma 2.0 on Vertex AI"""
    try:
        generation_config = GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens,
            top_p=top_p
        )

        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )

        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response."

print("✅ Helper functions defined")


✅ Successfully initialized projects/991182027087/locations/asia-southeast1/models/gemma-2-2b-it-1742823574849 on Vertex AI
✅ Helper functions defined


## Application 1: Structured Information Extraction

In [48]:
# @markdown Extract structured data from text using Gemma 2.0 on Vertex AI

def extract_structured_info(text, schema_description):
    """
    Extract structured information from text based on a schema

    Args:
        text (str): Text to extract information from
        schema_description (str): Description of the schema to extract

    Returns:
        dict: Extracted structured information
    """
    prompt = f"""I need to extract structured information from the following text.

Text: "{text}"

Please extract the following information:
{schema_description}

Return your answer as a valid JSON object only. No additional explanation needed.
"""

    response = generate_response(prompt, temperature=0.1)

    # Extract JSON from response
    try:
        # Find JSON-like structure in response
        start_idx = response.find('{')
        end_idx = response.rfind('}')

        if start_idx != -1 and end_idx != -1:
            json_str = response[start_idx:end_idx+1]
            return json.loads(json_str)
        else:
            # If no JSON markers found, try parsing the whole response
            return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON in response", "raw_response": response}

# Example: Extract event details
event_text = """
GDG DevFest Hanoi 2024 will be held on December 14th at the National Convention Center.
The Google Developer Groups are organizing this technology conference focusing on AI,
Mobile, Web, and Cloud technologies. Expected attendance is approximately 1,200
developers and tech enthusiasts from across the region. Registration is open until
December 1st, with tickets priced at $50 for regular admission and $25 for students.
"""

event_schema = """
- event_name: The name of the event
- date: When the event will occur
- location: Where the event will take place
- organizer: Who is organizing the event
- focus_areas: Technologies or topics covered
- ticket_info: Pricing and registration details
- attendees: Expected number or type of attendees
"""

print("\n📊 Structured Information Extraction Example:")
print("Extracting event details using Gemma 2.0 on Vertex AI...\n")

event_details = extract_structured_info(event_text, event_schema)
print("Extracted Event Details:")
print(json.dumps(event_details, indent=2))



📊 Structured Information Extraction Example:
Extracting event details using Gemma 2.0 on Vertex AI...

Error generating response: Invalid request.
Some of the fields of the request message are either not initialized or initialized with an invalid value.
Please make sure your request matches at least one accepted HTTP binding.
To match a binding the request message must have all the required fields initialized with values matching their patterns as listed below:
	URI: "/v1/{model=projects/*/locations/*/endpoints/*}:generateContent"
	Required request fields:
		field: "model", pattern: "projects/*/locations/*/endpoints/*"

	URI: "/v1/{model=projects/*/locations/*/publishers/*/models/*}:generateContent"
	Required request fields:
		field: "model", pattern: "projects/*/locations/*/publishers/*/models/*"

	URI: "/v1/{model=endpoints/*}:generateContent"
	Required request fields:
		field: "model", pattern: "endpoints/*"

	URI: "/v1/{model=publishers/*/models/*}:generateContent"
	Required reque

## Application 2: Multi-agent Research System powered by Gemma 2.0